**Pedro Henrique Mendes de Oliveira** - 171020091








 Regressão é uma técnica usada para predição de valores numéricos, mas também nos da informações sobre a relação entre os dados de entrada (features) e a variável de resposta. De modo geral se começa com um banco de dados com informações numéricas sobre alguma coisa. Tomando de exemplo casas, poderíamos ter dados sobre o tamanho, numero de quartos e banheiros, distancia até serviços públicos, quantos andares tem, ano em que foi construída e seu preço atual. Com métodos de regressão podemos descobrir a relação entre essas variáveis. E mesmo que alguma delas sejam obvias, por exemplo: Não precisamos de uma IA para nos dizer que casas maiores costumam custar mais caro, o interessante é entender o quanto cada variável afeta a outra. Em um estudo encontraram uma relação não linear entre o preço de casas e a distancia até um shopping. De acordo com o pesquisador casas vizinhas ao shopping tem seu valor diminuído, mas na faixa de 1 a 2 kilometros de distancia elas valorizam em grande escala e em seguida conforme a distancia aumenta o preço volta a diminuir.


 Podemos também realizar predições sobre novos dados usando essas técnicas. No nosso exemplo seria interessante prever por quanto uma casa pode ser vendida. Para realizar esta etapa é necessário dividir o banco de dados em duas partes, uma de treino e outra de teste. O modelo então é treinado usando apenas os exemplos de treino e em seguida é usado para predizer uma variável alvo dos exemplos de teste. Predições nunca serão exatas, a menos que trabalhemos em um espaço completamente determinístico, por isso é comumente usado métricas para determinar a imprecisão dos modelos, como a média do erro absoluto. Essa métrica consiste em comparar o valor que o modelo prediz para a variável alvo dos exemplos de teste com o seu valor real, então embora não teremos uma predição exata podemos saber o quão próximo, em media, o modelo chega dela. 

In [29]:
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
from sklearn import metrics, svm
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.neural_network import MLPRegressor

In [30]:
import time
from datetime import datetime
start_time = time.time()

Para este trabalho irei usar uma database sobre vinhos que contem vários dados numéricos e uma nota associada a sua qualidade. Os seguintes modelos serão utilizados:

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html

https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html

https://scikit-learn.org/stable/modules/generated/sklearn.gaussian_process.GaussianProcessRegressor.html

https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html

In [31]:
wine = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv', sep=';')

In [32]:
wine

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [33]:
X = wine.drop('quality',axis=1)
y = wine['quality']

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=777)

In [35]:
X_train = scale(X_train)
y_train = scale(y_train)

X_test = scale(X_test)
y_test = scale(y_test)

In [36]:
%%time
regressorSGD = SGDRegressor(loss='squared_error', learning_rate='constant', eta0=0.01, max_iter=1000, tol=1e-3).fit(X_train, y_train)

CPU times: user 2.53 ms, sys: 0 ns, total: 2.53 ms
Wall time: 2.63 ms


In [37]:
%%time
regressorSVM = svm.SVR().fit(X_train, y_train)

CPU times: user 129 ms, sys: 0 ns, total: 129 ms
Wall time: 132 ms


In [38]:
%%time
regressorGPR = GaussianProcessRegressor(random_state=0).fit(X_train, y_train)

CPU times: user 266 ms, sys: 98 ms, total: 364 ms
Wall time: 212 ms


In [39]:
%%time
regressorMLP = MLPRegressor(random_state=1, max_iter=5000).fit(X_train, y_train)

CPU times: user 6.58 s, sys: 5.56 s, total: 12.1 s
Wall time: 6.31 s


O modelo baseado no Gradient Descent é de longe o mais rapido, enquanto que o modelo baseado em redes neurais é muito mais lento que os outros. 

In [40]:
predSGD = regressorSGD.predict(X_test)
predSVM = regressorSVM.predict(X_test)
predMLP = regressorMLP.predict(X_test)
predGPR = regressorGPR.predict(X_test)

In [41]:
print('-------------------------------------------------')
print('SGD MAE:', metrics.mean_absolute_error(y_test, predSGD))
print('SGD MSE:', metrics.mean_squared_error(y_test, predSGD))
print('-------------------------------------------------')
print('SVM MAE:', metrics.mean_absolute_error(y_test, predSVM))
print('SVM MSE:', metrics.mean_squared_error(y_test, predSVM))
print('-------------------------------------------------')
print('Gaussian MAE:', metrics.mean_absolute_error(y_test, predGPR))
print('Gaussian MSE:', metrics.mean_squared_error(y_test, predGPR))
print('-------------------------------------------------')
print('Redes Neurais MAE:', metrics.mean_absolute_error(y_test, predMLP))
print('Redes Neurais MSE:', metrics.mean_squared_error(y_test, predMLP))


-------------------------------------------------
SGD MAE: 0.6197390378607425
SGD MSE: 0.6479402573250644
-------------------------------------------------
SVM MAE: 0.5509194250785825
SVM MSE: 0.5638755766351288
-------------------------------------------------
Gaussian MAE: 0.5819466372562907
Gaussian MSE: 0.6744125928250444
-------------------------------------------------
Redes Neurais MAE: 0.5938145201215556
Redes Neurais MSE: 0.5674287128056484


Para este dataset o modelo mais complexo apesar de ter se saido bem não foi o que retornou os melhores resultados, ficando atrás do Support Vector Regression.